In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:

def load_rnafm(filename, seq_len, L_max):
    """
    Load data from a .npy file and convert it to an N x N matrix.

    Parameters:
    - filename: Path to the .npy file.
    - N: Dimension of the square matrix.

    Returns:
    - bpp_matrix: N x N matrix reconstructed from the input file.
    """
    # Load the structured array from the .npy file
    data = np.load(filename)

    # Create an empty N x N matrix
    bpp_matrix = np.zeros((seq_len, seq_len))

    # Fill the matrix with the probabilities from the loaded data
    bpp_matrix[data["pos_1"], data["pos_2"]] = data["probabilities"]

    bpp_matrix = bpp_matrix + bpp_matrix.T - np.diag(np.diag(bpp_matrix))
    full = np.zeros((L_max, L_max))
    full[:seq_len, :seq_len] = bpp_matrix
    return torch.tensor(full)


def generate_base_pair_matrixv1(file_path, L):
    """
    Reads a TXT file of base pair probabilities and generates an n x n matrix.

    Args:
    - file_path (str): Path to the TXT file.

    Returns:
    - np.array: An n x n matrix of base pair probabilities.
    """
    # Read the data using pandas
    data = pd.read_csv(file_path, sep=" ", header=None, names=["pos1", "pos2", "prob"])

    # Find the largest position in the 'pos1' column
    largest_position = data["pos1"].max()

    ids = torch.from_numpy(data[["pos1", "pos2"]].values.astype(int))
    matrix = torch.zeros((L, L))
    matrix[ids[:, 0] - 1, ids[:, 1] - 1] = torch.from_numpy(data["prob"].values).float()
    matrix[ids[:, 1] - 1, ids[:, 0] - 1] = torch.from_numpy(data["prob"].values).float()

    return matrix


def extra_bpp_from_numpy(filename, N, seq_len=None):
    """
    Load data from a .npy file and convert it to an N x N matrix.

    Parameters:
    - filename: Path to the .npy file.
    - N: Dimension of the square matrix.

    Returns:
    - bpp_matrix: N x N matrix reconstructed from the input file.
    """
    # Load the structured array from the .npy file
    if filename.parent.stem in ["rnafm", "rnaformerv1"]:
        full = load_rnafm(filename, seq_len, N)
    else:
        data = np.load(filename)
        # Create an empty N x N matrix
        bpp_matrix = np.zeros((N, N))
        # Fill the matrix with the probabilities from the loaded data
        bpp_matrix[data["pos_1"], data["pos_2"]] = data["probabilities"]
        full = torch.tensor(bpp_matrix)

    return full

def dot_to_adjacencyv0(dot_notation, n):
    adjacency_matrix = np.zeros((n, n), dtype=int)
    stack = []
    for i, char in enumerate(dot_notation):
        if char == "(":
            stack.append(i)
        elif char == ")":
            j = stack.pop()
            adjacency_matrix[i][j] = adjacency_matrix[j][i] = 1

    return adjacency_matrix

In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
bpp_df.drop_duplicates(subset = 'sequence_id', inplace=True)
df = pd.concat([pd.read_parquet('../eda/train_ss_vienna_rna.parquet'),
                pd.read_parquet('../eda/test_ss_vienna_rna.parquet')])
df = df.drop_duplicates(subset=['sequence_id']).reset_index(drop=True)
df = pd.merge(df, bpp_df, on='sequence_id')
df['L'] = df['sequence'].apply(len)




In [4]:
# OUT = Path('bpp/comb')
# os.makedirs(OUT, exist_ok=True)


In [ ]:
# def convert(row):
#     L = row.L
#     bpp_fn = row.bpp  # You might need to convert this to Path object if it's not already.
#     ss_full = row.ss_full

#     extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv1", "rnaformer"]
#     extra_bpp_path = Path("bpp")
#     names = ['vienna_2', 'contrafold_2', 'rnaformerv1', "rnaformer", 'bpp_org', 'ss_vienna']

#     bpp_extra = [
#         extra_bpp_from_numpy(extra_bpp_path / f"{i}/{bpp_fn.stem}.npy", L, seq_len=L).numpy()
#         for i in extra_bpp
#     ] + [generate_base_pair_matrixv1(bpp_fn, L).numpy().astype(np.float16)] + [dot_to_adjacencyv0(ss_full, L)]

#     bpp_extra_d = {s : np.array(d).astype(np.float16) for s, d in zip(names, bpp_extra)}

#     # Assuming OUT is a predefined path
#     OUT = Path('bpp/comb')  # replace with your actual output path
#     np.savez_compressed(OUT / f"{row.sequence_id}.npz", **bpp_extra_d)

# # This function will be used by Pool.map which expects a single-argument function
# def worker(index):
#     row = df.iloc[index]
#     convert(row)
    
# with Pool(processes=cpu_count()) as pool:
#     max_ = df.shape[0]
#     with tqdm(total=max_) as pbar:
#         for _ in pool.imap_unordered(worker, range(max_)):
#             pbar.update()

In [5]:
def get_original_dict(row):
    L = row.L
    bpp_fn = row.bpp  # You might need to convert this to Path object if it's not already.
    ss_full = row.ss_full

    extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv1", "rnaformer"]
    extra_bpp_path = Path("bpp")
    names = ['vienna_2', 'contrafold_2', 'rnaformerv1', "rnaformer", 'bpp_org', 'ss_vienna']

    bpp_extra = [
        extra_bpp_from_numpy(extra_bpp_path / f"{i}/{bpp_fn.stem}.npy", L, seq_len=L).numpy()
        for i in extra_bpp
    ] + [generate_base_pair_matrixv1(bpp_fn, L).numpy().astype(np.float16)] + [dot_to_adjacencyv0(ss_full, L)]

    bpp_extra_d = {s : np.array(d).astype(np.float16) for s, d in zip(names, bpp_extra)}
    return bpp_extra_d

def get_saved(row):
    L = row.L
    bpp_fn = row.bpp  # You might need to convert this to Path object if it's not already.
    ss_full = row.ss_full

    extra_bpp = ["vienna_2", "contrafold_2", "rnaformerv1", "rnaformer"]
    OUT = Path('bpp/comb')  # replace with your actual output path
    data = np.load(OUT / f"{row.sequence_id}.npz")
    return data

In [6]:
index = 23
row = df.iloc[index]

In [ ]:
get_saved(row)

In [ ]:
f